In [189]:
import requests
import datetime
import json
import pytz

from urllib.parse import urljoin 
from getpass import getpass
from requests.auth import HTTPBasicAuth
from pprint import pprint

Pretty printing has been turned ON


In [ ]:
# Turn on pretty printing
%pprint

The Ovation web API is a RESTful API that you use with any HTTP client. In this tutorial, we'll use the Python `requests` package.

### Utilities

In [23]:
API_HOST = 'http://127.0.0.1:3000' #'https://api-dev.ovation.io/'
def make_url(path, host=API_HOST):
    return urljoin(host, path)

In [14]:
make_url('/api/v1/projects')

'http://0.0.0.0:3000/api/v1/projects'

### Credentials

Enter your Ovation API key:

In [16]:
api_key = getpass(prompt="API Key: ")

API Key: ········


We create a `requests.Session` object for use with the Ovation API

In [34]:
def make_session(api_key=api_key):
    auth_params = {'api-key' : api_key}
    
    api = requests.Session()
    api.verify = True # Verify SSL certificates
    api.headers['content-type'] = 'application/json'
    api.headers['authorization'] = 'Token token={}'.format(api_key)
    
    return api

api = make_session(api_key)

### Projects

In [77]:
make_url('/api/v1/projects')

'http://127.0.0.1:3000/api/v1/projects'

In [174]:
r = api.get(make_url('/api/v1/projects'))
r.raise_for_status()

In [175]:
projects = r.json()['projects']

In [123]:
len(projects)

10

In [192]:
pprint(projects[0])

{'_id': '52b6366a-1528-4bf9-a091-09c4e935d1cd',
 '_rev': '3-eec6d9c40436a57aca7d88e96f92cb54',
 'api_version': 4,
 'attributes': {'name': 'abby'},
 'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
           'analysis_records': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/analysis_records',
           'experiments': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/experiments',
           'notes': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/notes',
           'owner': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/owner',
           'properties': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/properties',
           'self': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd',
           'sftp': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/sftp',
           'tags': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/tags',
           'timeline-events': '/api/v

In [193]:
pprint(projects[1])

{'_id': '647ee703-d6ee-44ba-ac9f-a9e6c87e3301',
 '_rev': '17-caee57966eb8f2cda9e68bb534ed71aa',
 'api_version': 4,
 'attributes': {'name': 'sftp-upload-test'},
 'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd',
                                    '647ee703-d6ee-44ba-ac9f-a9e6c87e3301'],
           'analysis_records': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/analysis_records',
           'experiments': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/experiments',
           'notes': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/notes',
           'owner': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/owner',
           'properties': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/properties',
           'self': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301',
           'tags': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/tags',
           'timeline-events': '/api

## Creating links

In [157]:
links = [
  {"target_id": projects[1]['_id']}
]
r = api.post(make_url('/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/sftp'), data=json.dumps(links))
r.raise_for_status()
pprint(r.json())

{'entities': [{'_id': '52b6366a-1528-4bf9-a091-09c4e935d1cd',
               '_rev': '2-69dbfca3c25590f089e9bcf2c6ccedc8',
               'api_version': 4,
               'attributes': {'name': 'abby'},
               'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
                         'analysis_records': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/analysis_records',
                         'experiments': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/experiments',
                         'notes': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/notes',
                         'owner': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/owner',
                         'properties': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/properties',
                         'self': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd',
                         'sftp': '/api/v1/entities/52b636

### Following links

In [177]:
r = api.get(make_url(projects[0]['links']['sftp']))
r.raise_for_status()
targets = r.json()['sftp']
pprint(targets)

[{'_id': '647ee703-d6ee-44ba-ac9f-a9e6c87e3301',
  '_rev': '17-caee57966eb8f2cda9e68bb534ed71aa',
  'api_version': 4,
  'attributes': {'name': 'sftp-upload-test'},
  'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd',
                                     '647ee703-d6ee-44ba-ac9f-a9e6c87e3301'],
            'analysis_records': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/analysis_records',
            'experiments': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/experiments',
            'notes': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/notes',
            'owner': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/owner',
            'properties': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/properties',
            'self': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301',
            'tags': '/api/v1/entities/647ee703-d6ee-44ba-ac9f-a9e6c87e3301/links/tags',
            'timeline-

### Adding analysis record

In [195]:
# Attributes of the analysis -- we'll fill these in after creation
est = pytz.timezone('US/Eastern')
d = datetime.datetime.now(est)
attributes = {'name': 'Tutorial Analysis',
              'date': d.isoformat()}

# Analysis parameters —- what's needed to recreate the analysis. This can be anything, but you may want to include
# code repository (and revision), algorithm parameters, etc.
parameters = {'code': 'https://github.com/physion/ovation-python',
             'revision': 'c62e11e', # e.g. Git hash
              'algorithm parameters' : {'param1': 1}}

new_record = {'parameters': parameters,
             'inputs': [],
             'outputs': []}

r = api.post(make_url('/api/v1/analysis_records'), data=json.dumps(new_record))
pprint(r.json())

ValueError: Expecting value: line 1 column 1 (char 0)

In [93]:
ar = r.json()[0]

In [66]:
r.status_code

201

Refresh projects so that we get the up-to-date links

In [77]:
projects[0]['links']['self']

'/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6'

In [84]:
p = api.get(make_url(projects[0]['links']['self'])).json()['entity']

In [85]:
p

{'_id': '7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6',
 'named_links': {},
 'attributes': {'name': 'My First Project',
  'purpose': 'First Project',
  'start': '2014-09-25T22:18:19.724-04:00'},
 'type': 'Project',
 'links': {'experiments': '/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6/links/experiments',
  'self': '/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6',
  'owner': '/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6/links/owner',
  '_collaboration_roots': '/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6/links/_collaboration_roots'},
 '_rev': '3-b0ede23a375c31c56663960447319237'}

In [94]:
api.get(make_url(ar['links']['self'])).json()

[{'_id': '0fd8fd6e-d996-4595-a547-745e949c98af',
  'links': {'self': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af',
   'parent': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af/links/parent',
   'owner': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af/links/owner',
   '_collaboration_roots': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af/links/_collaboration_roots'},
  'attributes': {'name': 'Tutorial Analysis',
   'date': '2014-10-10T13:32:26.077167-04:00'},
  'type': 'AnalysisRecord',
  '_rev': '1-bd7b00dcd33ed2cb8a387745b6133896'}]

In [98]:
# TODO p['links']['analysis_records']
api.get(make_url('/api/v1/entities/{}/links/analysis_records'.format(p['_id']))).json()

[]

'7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6'